In [1]:
from dotenv import load_dotenv
load_dotenv(override=True)

True

In [2]:
import os
import tempfile
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders.sitemap import SitemapLoader
from langchain_community.vectorstores import SKLearnVectorStore
from langchain_openai import OpenAIEmbeddings
from langsmith import traceable
from openai import OpenAI
from typing import List
import nest_asyncio

MODEL_NAME = "gpt-4o"
MODEL_PROVIDER = "openai"
APP_VERSION = 1.0
RAG_SYSTEM_PROMPT = """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the latest question in the conversation. 
If you don't know the answer, just say that you don't know. 
Use three sentences maximum and keep the answer concise.
"""

openai_client = OpenAI()

def get_vector_db_retriever():
    persist_path = os.path.join(tempfile.gettempdir(), "union.parquet")
    embd = OpenAIEmbeddings()

    if os.path.exists(persist_path):
        vectorstore = SKLearnVectorStore(
            embedding=embd,
            persist_path=persist_path,
            serializer="parquet"
        )
        return vectorstore.as_retriever(lambda_mult=0)

    ls_docs_sitemap_loader = SitemapLoader(web_path="https://docs.smith.langchain.com/sitemap.xml", continue_on_failure=True)
    ls_docs = ls_docs_sitemap_loader.load()

    text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
        chunk_size=500, chunk_overlap=0
    )
    doc_splits = text_splitter.split_documents(ls_docs)

    vectorstore = SKLearnVectorStore.from_documents(
        documents=doc_splits,
        embedding=embd,
        persist_path=persist_path,
        serializer="parquet"
    )
    vectorstore.persist()
    return vectorstore.as_retriever(lambda_mult=0)

nest_asyncio.apply()
retriever = get_vector_db_retriever()

"""
retrieve_documents
- Returns documents fetched from a vectorstore based on the user's question
"""
@traceable(run_type="chain")
def retrieve_documents(question: str):
    return retriever.invoke(question)

"""
generate_response
- Calls `call_openai` to generate a model response after formatting inputs
"""
@traceable(run_type="chain")
def generate_response(question: str, documents):
    formatted_docs = "\n\n".join(doc.page_content for doc in documents)
    messages = [
        {
            "role": "system",
            "content": RAG_SYSTEM_PROMPT
        },
        {
            "role": "user",
            "content": f"Context: {formatted_docs} \n\n Question: {question}"
        }
    ]
    return call_openai(messages)

"""
call_openai
- Returns the chat completion output from OpenAI
"""
@traceable(
    run_type="llm",
    metadata={
        "ls_provider": MODEL_PROVIDER,
        "ls_model_name": MODEL_NAME
    }
)
def call_openai(messages: List[dict]) -> str:
    return openai_client.chat.completions.create(
        model=MODEL_NAME,
        messages=messages,
    )

"""
langsmith_rag
- Calls `retrieve_documents` to fetch documents
- Calls `generate_response` to generate a response based on the fetched documents
- Returns the model response
"""
@traceable(run_type="chain")
def langsmith_rag(question: str):
    documents = retrieve_documents(question)
    response = generate_response(question, documents)
    return response.choices[0].message.content

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [3]:
from langsmith import evaluate, Client

client = Client()
dataset_name = "Electronics dataset"

def is_concise_enough(reference_outputs: dict, outputs: dict) -> dict:
    score = len(outputs["output"]) < 1.5 * len(reference_outputs["output"])
    return {"key": "is_concise", "score": int(score)}

def target_function(inputs: dict):
    return langsmith_rag(inputs["question"])

evaluate(
    target_function,
    data=dataset_name,
    evaluators=[is_concise_enough],
    experiment_prefix="gpt-4o"
)

/opt/miniconda3/envs/mat/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


View the evaluation results for experiment: 'gpt-4o-81c3b6ae' at:
https://smith.langchain.com/o/bcaa99f9-3a04-43e7-9c9d-c034ad1da1ea/datasets/7b1f15f2-1b85-4f53-b26c-70e6c86dd825/compare?selectedSessions=449bb12f-af21-4113-8f81-57a320b4854c




10it [00:28,  2.80s/it]


,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,How do I set up I2C communication between Ardu...,"I'm sorry, but I don't have information on set...",None,"To set up I2C communication on Arduino, use th...",1,2.058623,4bd00e46-d26d-4835-993c-a814e7b0676f,0ebd6bf3-8d92-40e0-9c5c-d264c4de394f
1,What's the best way to power Arduino projects?,The best way to power Arduino projects is typi...,None,"For Arduino projects, you can power via USB (5...",1,2.850456,5ba7d138-0861-4c97-acae-90d661e0705a,8e8f0b98-b5dc-4c80-bbba-40fe0c47e3b2
2,How do I read analog sensor values on Arduino?,"To read analog sensor values on an Arduino, yo...",None,"To read analog sensor values on Arduino, conne...",1,3.908915,6d44f1ba-a2c4-4e8e-a60b-92de76c05143,282cba81-c721-46ad-acc8-7e00b9dfe971
3,How do I create a simple IoT project with Rasp...,To create a simple IoT project with Raspberry ...,None,"For a simple IoT project with Raspberry Pi, st...",1,2.948872,9a0a4607-2b8b-47ea-a977-67a4360d9eec,383b8b8c-eda6-40a9-9424-0e5a7eb0969e
4,What's the difference between Arduino Uno and ...,The Arduino Uno is a microcontroller board opt...,None,Arduino Uno is a microcontroller board best fo...,1,2.893398,9d937a05-4c2d-49a4-9c53-c37c10df9c61,3f1fd12c-6935-4acd-9c07-0fcbf57b0037
5,What are common mistakes beginners make with A...,Common mistakes beginners make with Arduino in...,None,Common Arduino mistakes include: forgetting cu...,1,2.342701,a6f8deda-8043-47e1-b83c-6927398e288b,93e1687c-aafb-4014-acd1-506343349a3a
6,How do I control a servo motor with Raspberry Pi?,"To control a servo motor with a Raspberry Pi, ...",None,"To control a servo motor with Raspberry Pi, co...",1,3.067265,b9bfaa82-8932-409b-8b96-c740de91ee67,b993c527-33f9-492d-a99e-2556b370b3ba
7,How do I set up a basic LED circuit with Ardui...,To set up a basic LED circuit with an Arduino ...,None,To set up a basic LED circuit with Arduino Uno...,1,2.143839,c4b3276a-2a1d-4a12-a688-8e116ad23b7b,eee3e5d5-cf36-43e9-9790-94ab23f3e4c3
8,What are the best sensors to start with for Ar...,I don't have information on the best sensors f...,None,The best beginner sensors for Arduino include:...,1,2.245312,ea00c58e-74a2-4de1-8396-353e25aa4b9e,dfd755a5-eb5f-4419-9605-609dbb960520
9,What programming languages can I use with Rasp...,You can use a variety of programming languages...,None,Raspberry Pi supports multiple programming lan...,1,3.019497,f8c359e3-1514-41a6-befd-ca15aa56b3cb,056945ce-b2c6-4745-8d79-723c486ca8dc


In [4]:
from langsmith import evaluate, Client
from langsmith.schemas import Example, Run

def target_function(inputs: dict):
    return langsmith_rag(inputs["question"])

evaluate(
    target_function,
    data=dataset_name,
    evaluators=[is_concise_enough],
    experiment_prefix="gpt-3.5-turbo"
)

View the evaluation results for experiment: 'gpt-3.5-turbo-6b7d76ed' at:
https://smith.langchain.com/o/bcaa99f9-3a04-43e7-9c9d-c034ad1da1ea/datasets/7b1f15f2-1b85-4f53-b26c-70e6c86dd825/compare?selectedSessions=67e084a6-150c-4f56-9379-37a2db476171




10it [00:29,  3.00s/it]


,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,How do I set up I2C communication between Ardu...,"I'm sorry, but I don't have information on set...",None,"To set up I2C communication on Arduino, use th...",1,2.539194,4bd00e46-d26d-4835-993c-a814e7b0676f,f32ccaee-cd66-4eeb-bc82-3cd21120a97e
1,What's the best way to power Arduino projects?,The best way to power Arduino projects is typi...,None,"For Arduino projects, you can power via USB (5...",1,3.598182,5ba7d138-0861-4c97-acae-90d661e0705a,79c0d407-ff50-4a15-9a00-5e9087f05667
2,How do I read analog sensor values on Arduino?,"To read analog sensor values on an Arduino, yo...",None,"To read analog sensor values on Arduino, conne...",1,2.643336,6d44f1ba-a2c4-4e8e-a60b-92de76c05143,5d875dd4-7fde-4dea-9644-3e3b46f065be
3,How do I create a simple IoT project with Rasp...,I don't have information on creating an IoT pr...,None,"For a simple IoT project with Raspberry Pi, st...",1,2.521245,9a0a4607-2b8b-47ea-a977-67a4360d9eec,ed19e30d-913a-4c3d-bae4-3c4ca1e1367f
4,What's the difference between Arduino Uno and ...,The Arduino Uno is a microcontroller board use...,None,Arduino Uno is a microcontroller board best fo...,1,3.179856,9d937a05-4c2d-49a4-9c53-c37c10df9c61,879626a6-7e04-47a5-b19a-45687542ad9c
5,What are common mistakes beginners make with A...,Common mistakes beginners make with Arduino in...,None,Common Arduino mistakes include: forgetting cu...,1,1.873941,a6f8deda-8043-47e1-b83c-6927398e288b,fc1de8fe-e9bf-49b1-bc49-9fea06cdca2a
6,How do I control a servo motor with Raspberry Pi?,"To control a servo motor with a Raspberry Pi, ...",None,"To control a servo motor with Raspberry Pi, co...",1,3.745045,b9bfaa82-8932-409b-8b96-c740de91ee67,5d565e34-e202-47ba-a612-54793975c6ee
7,How do I set up a basic LED circuit with Ardui...,To set up a basic LED circuit with an Arduino ...,None,To set up a basic LED circuit with Arduino Uno...,1,5.854187,c4b3276a-2a1d-4a12-a688-8e116ad23b7b,b32a5839-1d40-471c-b4e9-964a5930bfd4
8,What are the best sensors to start with for Ar...,I don't know.,None,The best beginner sensors for Arduino include:...,1,1.208881,ea00c58e-74a2-4de1-8396-353e25aa4b9e,7a8417cd-27e0-4cc3-9d18-3cb49f5f2e21
9,What programming languages can I use with Rasp...,You can use a variety of programming languages...,None,Raspberry Pi supports multiple programming lan...,1,2.296644,f8c359e3-1514-41a6-befd-ca15aa56b3cb,e5c5e80b-7fb3-4c56-861d-3db3b033d06a


In [7]:
evaluate(
    target_function,
    data=client.list_examples(dataset_name=dataset_name, as_of="initial dataset"),   # We use as_of to specify a version
    evaluators=[is_concise_enough],
    experiment_prefix="initial dataset"
)

View the evaluation results for experiment: 'initial dataset-94395f68' at:
https://smith.langchain.com/o/bcaa99f9-3a04-43e7-9c9d-c034ad1da1ea/datasets/7b1f15f2-1b85-4f53-b26c-70e6c86dd825/compare?selectedSessions=87cfc1c4-8c3b-4287-9191-cb450c81c492




10it [00:27,  2.73s/it]


,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,How do I set up I2C communication between Ardu...,I don't know.,None,"To set up I2C communication on Arduino, use th...",1,1.246319,4bd00e46-d26d-4835-993c-a814e7b0676f,53fb19f0-2295-48cf-9470-cbf98b729a54
1,What's the best way to power Arduino projects?,The best way to power Arduino projects is typi...,None,"For Arduino projects, you can power via USB (5...",1,4.806926,5ba7d138-0861-4c97-acae-90d661e0705a,21580e6f-b50a-4e45-8d4c-15121ec237ed
2,How do I read analog sensor values on Arduino?,"To read analog sensor values on an Arduino, yo...",None,"To read analog sensor values on Arduino, conne...",1,2.812338,6d44f1ba-a2c4-4e8e-a60b-92de76c05143,702aedbf-37a8-4554-badd-ad2e67c69e42
3,How do I create a simple IoT project with Rasp...,I don't have the information on how to create ...,None,"For a simple IoT project with Raspberry Pi, st...",1,2.746190,9a0a4607-2b8b-47ea-a977-67a4360d9eec,4218c18a-4753-47eb-8f57-424b0af09fc3
4,What's the difference between Arduino Uno and ...,The Arduino Uno is a microcontroller board des...,None,Arduino Uno is a microcontroller board best fo...,1,3.114524,9d937a05-4c2d-49a4-9c53-c37c10df9c61,e8c3d3be-25ff-4ac2-8083-afad4a83467f
5,What are common mistakes beginners make with A...,Common mistakes beginners make with Arduino in...,None,Common Arduino mistakes include: forgetting cu...,1,1.999317,a6f8deda-8043-47e1-b83c-6927398e288b,1275e748-1286-4cb6-8ec4-ddddca31f0b5
6,How do I control a servo motor with Raspberry Pi?,"To control a servo motor with a Raspberry Pi, ...",None,"To control a servo motor with Raspberry Pi, co...",1,2.594450,b9bfaa82-8932-409b-8b96-c740de91ee67,57527542-66fa-4180-9894-2c91ac83d7ee
7,How do I set up a basic LED circuit with Ardui...,To set up a basic LED circuit with an Arduino ...,None,To set up a basic LED circuit with Arduino Uno...,1,2.364024,c4b3276a-2a1d-4a12-a688-8e116ad23b7b,2743da86-0a9a-4c22-b946-b6c1cd9048c9
8,What are the best sensors to start with for Ar...,I don't have specific recommendations for the ...,None,The best beginner sensors for Arduino include:...,1,2.261732,ea00c58e-74a2-4de1-8396-353e25aa4b9e,490f04aa-7cfc-4304-9789-c1e3bf0526b9
9,What programming languages can I use with Rasp...,You can use several programming languages with...,None,Raspberry Pi supports multiple programming lan...,1,2.721297,f8c359e3-1514-41a6-befd-ca15aa56b3cb,669b9478-37bc-4dd5-b59c-7597ae5561f0


In [12]:
evaluate(
    target_function,
    data=client.list_examples(dataset_name=dataset_name, splits=["Crucial Example split"]),
    evaluators=[is_concise_enough],
    experiment_prefix="Crucial Example split"
)

View the evaluation results for experiment: 'Crucial Example split-68564236' at:
https://smith.langchain.com/o/bcaa99f9-3a04-43e7-9c9d-c034ad1da1ea/datasets/7b1f15f2-1b85-4f53-b26c-70e6c86dd825/compare?selectedSessions=fca45bdd-98bd-41b0-85cc-98928ab1caa1




5it [00:17,  3.51s/it]


,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,How do I set up I2C communication between Ardu...,I don't know.,None,"To set up I2C communication on Arduino, use th...",1,1.665875,4bd00e46-d26d-4835-993c-a814e7b0676f,2db8e18d-92cd-4bbd-90f2-c97c28bc7a8a
1,What's the best way to power Arduino projects?,The best way to power Arduino projects depends...,None,"For Arduino projects, you can power via USB (5...",1,3.016899,5ba7d138-0861-4c97-acae-90d661e0705a,9832dc2f-e818-4c98-a55f-d2661f19a72f
2,How do I read analog sensor values on Arduino?,"To read analog sensor values on Arduino, use t...",None,"To read analog sensor values on Arduino, conne...",1,2.782818,6d44f1ba-a2c4-4e8e-a60b-92de76c05143,0a8eb2f1-4334-42e7-8806-6814cf052a75
3,How do I create a simple IoT project with Rasp...,I don't have information on creating a simple ...,None,"For a simple IoT project with Raspberry Pi, st...",1,5.506481,9a0a4607-2b8b-47ea-a977-67a4360d9eec,17eb5136-774d-4ff5-b5f4-b9ce73290ee6
4,What's the difference between Arduino Uno and ...,The Arduino Uno is a microcontroller board use...,None,Arduino Uno is a microcontroller board best fo...,1,3.985075,9d937a05-4c2d-49a4-9c53-c37c10df9c61,835e65f6-c0b5-4cbb-a033-473fd0480423


In [13]:
evaluate(
    target_function,
    data=client.list_examples(
        dataset_name=dataset_name, 
        example_ids=[ 
            "f8c359e3-1514-41a6-befd-ca15aa56b3cb",
            "ea00c58e-74a2-4de1-8396-353e25aa4b9e"
        ]
    ),
    evaluators=[is_concise_enough],
    experiment_prefix="two specific example ids"
)

View the evaluation results for experiment: 'two specific example ids-41bacbb9' at:
https://smith.langchain.com/o/bcaa99f9-3a04-43e7-9c9d-c034ad1da1ea/datasets/7b1f15f2-1b85-4f53-b26c-70e6c86dd825/compare?selectedSessions=9e05d8d7-fb11-4ae8-9e8c-d6f040957c30




2it [00:04,  2.43s/it]


,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,What are the best sensors to start with for Ar...,I don't have specific information on the best ...,None,The best beginner sensors for Arduino include:...,1,2.309456,ea00c58e-74a2-4de1-8396-353e25aa4b9e,c2c3d672-8e00-417f-a2ac-d9f0492cd1f4
1,What programming languages can I use with Rasp...,You can use programming languages such as Pyth...,None,Raspberry Pi supports multiple programming lan...,1,2.043028,f8c359e3-1514-41a6-befd-ca15aa56b3cb,1a2eb9b4-5902-493e-a795-c878b6549c1a


In [14]:
evaluate(
    target_function,
    data=dataset_name,
    evaluators=[is_concise_enough],
    experiment_prefix="two repetitions",
    num_repetitions=2
)

View the evaluation results for experiment: 'two repetitions-8d2899fe' at:
https://smith.langchain.com/o/bcaa99f9-3a04-43e7-9c9d-c034ad1da1ea/datasets/7b1f15f2-1b85-4f53-b26c-70e6c86dd825/compare?selectedSessions=6ff8195a-9db3-4ecd-b609-b217a1a755c6




20it [01:05,  3.27s/it]


,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,How do I set up I2C communication between Ardu...,I don't have that information. Please refer to...,None,"To set up I2C communication on Arduino, use th...",1,3.166634,4bd00e46-d26d-4835-993c-a814e7b0676f,b43c0893-05a9-422d-a745-34ed8b91ae14
1,What's the best way to power Arduino projects?,The best way to power Arduino projects typical...,None,"For Arduino projects, you can power via USB (5...",1,4.401567,5ba7d138-0861-4c97-acae-90d661e0705a,52451a19-6dc1-44a6-b72a-8e6ac66fb966
2,How do I read analog sensor values on Arduino?,"To read analog sensor values on an Arduino, us...",None,"To read analog sensor values on Arduino, conne...",1,2.437355,6d44f1ba-a2c4-4e8e-a60b-92de76c05143,813ffd01-8dd6-4559-b6e1-f145cbbf7682
3,How do I create a simple IoT project with Rasp...,To create a simple IoT project with a Raspberr...,None,"For a simple IoT project with Raspberry Pi, st...",1,3.426252,9a0a4607-2b8b-47ea-a977-67a4360d9eec,0c05aa02-bd79-4811-84be-37dedf631b06
4,What's the difference between Arduino Uno and ...,The Arduino Uno is a microcontroller board ide...,None,Arduino Uno is a microcontroller board best fo...,1,5.075630,9d937a05-4c2d-49a4-9c53-c37c10df9c61,4c9b5159-c3ae-44c9-87ba-8ca0cb99fad1
5,What are common mistakes beginners make with A...,Common mistakes beginners make with Arduino in...,None,Common Arduino mistakes include: forgetting cu...,1,3.772629,a6f8deda-8043-47e1-b83c-6927398e288b,bbb9f418-c062-4a0a-8950-fe1cc625e2ea
6,How do I control a servo motor with Raspberry Pi?,"To control a servo motor with a Raspberry Pi, ...",None,"To control a servo motor with Raspberry Pi, co...",1,2.106183,b9bfaa82-8932-409b-8b96-c740de91ee67,c3ce0235-4776-43e8-9c4e-8000e0880995
7,How do I set up a basic LED circuit with Ardui...,To set up a basic LED circuit with an Arduino ...,None,To set up a basic LED circuit with Arduino Uno...,1,4.333287,c4b3276a-2a1d-4a12-a688-8e116ad23b7b,7c98147b-0939-4dd5-b212-e09ea6dc2d49
8,What are the best sensors to start with for Ar...,I don't know.,None,The best beginner sensors for Arduino include:...,1,1.345861,ea00c58e-74a2-4de1-8396-353e25aa4b9e,d145be40-7c7f-42f9-9a12-34e6e052a613
9,What programming languages can I use with Rasp...,You can use several programming languages with...,None,Raspberry Pi supports multiple programming lan...,1,2.733317,f8c359e3-1514-41a6-befd-ca15aa56b3cb,3a62757e-159f-4850-ba3e-1df3944392a4


In [15]:
evaluate(
    target_function,
    data=dataset_name,
    evaluators=[is_concise_enough],
    experiment_prefix="concurrency",
    max_concurrency=3,
)

View the evaluation results for experiment: 'concurrency-4b2009d0' at:
https://smith.langchain.com/o/bcaa99f9-3a04-43e7-9c9d-c034ad1da1ea/datasets/7b1f15f2-1b85-4f53-b26c-70e6c86dd825/compare?selectedSessions=e6e8ab3f-5d41-482d-98f4-4e9aff06c050




10it [00:10,  1.08s/it]


,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,How do I set up I2C communication between Ardu...,I don't know.,None,"To set up I2C communication on Arduino, use th...",1,1.527986,4bd00e46-d26d-4835-993c-a814e7b0676f,f2931d14-04b0-429d-b8ee-4ce0b357f3e1
1,How do I read analog sensor values on Arduino?,"To read analog sensor values on an Arduino, us...",None,"To read analog sensor values on Arduino, conne...",1,3.273610,6d44f1ba-a2c4-4e8e-a60b-92de76c05143,eac37658-957a-4a23-a5dd-1fec7d168240
2,What's the best way to power Arduino projects?,The best way to power Arduino projects depends...,None,"For Arduino projects, you can power via USB (5...",1,3.863454,5ba7d138-0861-4c97-acae-90d661e0705a,8af79d99-de62-4d43-8d23-9da59748e454
3,How do I create a simple IoT project with Rasp...,To create a simple IoT project with a Raspberr...,None,"For a simple IoT project with Raspberry Pi, st...",1,2.516273,9a0a4607-2b8b-47ea-a977-67a4360d9eec,3279b0dd-c6c9-4a92-b68a-3c536b4828a5
4,What's the difference between Arduino Uno and ...,The Arduino Uno is a microcontroller board des...,None,Arduino Uno is a microcontroller board best fo...,1,3.432478,9d937a05-4c2d-49a4-9c53-c37c10df9c61,2a3beb36-a44b-486d-9d5f-abc610865a57
5,How do I control a servo motor with Raspberry Pi?,"To control a servo motor with a Raspberry Pi, ...",None,"To control a servo motor with Raspberry Pi, co...",1,2.877016,b9bfaa82-8932-409b-8b96-c740de91ee67,edc78804-fb40-461c-925f-2f7bd8abf653
6,What are common mistakes beginners make with A...,Common mistakes beginners make with Arduino in...,None,Common Arduino mistakes include: forgetting cu...,1,3.081943,a6f8deda-8043-47e1-b83c-6927398e288b,dd27c50d-f79b-410f-b71a-089bb5c21f7f
7,What are the best sensors to start with for Ar...,I don't know.,None,The best beginner sensors for Arduino include:...,1,1.236873,ea00c58e-74a2-4de1-8396-353e25aa4b9e,dfe7ab5a-e0df-43f5-81fe-b7b8b525b6f0
8,How do I set up a basic LED circuit with Ardui...,To set up a basic LED circuit with an Arduino ...,None,To set up a basic LED circuit with Arduino Uno...,1,3.092830,c4b3276a-2a1d-4a12-a688-8e116ad23b7b,a62a989d-fb12-49c7-8a52-c8895d0f1128
9,What programming languages can I use with Rasp...,You can use several programming languages with...,None,Raspberry Pi supports multiple programming lan...,1,3.274947,f8c359e3-1514-41a6-befd-ca15aa56b3cb,05cd753f-8a01-4a3e-97e3-520d0945a3a2


In [16]:
evaluate(
    target_function,
    data=dataset_name,
    evaluators=[is_concise_enough],
    experiment_prefix="metadata added",
    metadata={
        "model_name": MODEL_NAME
    }
)

View the evaluation results for experiment: 'metadata added-4fb7764d' at:
https://smith.langchain.com/o/bcaa99f9-3a04-43e7-9c9d-c034ad1da1ea/datasets/7b1f15f2-1b85-4f53-b26c-70e6c86dd825/compare?selectedSessions=fb58f9f3-4c81-446b-860a-2926c2fc744b




10it [00:27,  2.80s/it]


,inputs.question,outputs.output,error,reference.output,feedback.is_concise,execution_time,example_id,id
0,How do I set up I2C communication between Ardu...,I don't know.,None,"To set up I2C communication on Arduino, use th...",1,1.302007,4bd00e46-d26d-4835-993c-a814e7b0676f,2ebefbfb-522d-43f2-871d-b7ff9f309eb9
1,What's the best way to power Arduino projects?,The best way to power Arduino projects depends...,None,"For Arduino projects, you can power via USB (5...",1,2.894146,5ba7d138-0861-4c97-acae-90d661e0705a,1aec40f5-c995-46e6-9e93-8563a7eedcec
2,How do I read analog sensor values on Arduino?,"To read analog sensor values on an Arduino, yo...",None,"To read analog sensor values on Arduino, conne...",1,3.353580,6d44f1ba-a2c4-4e8e-a60b-92de76c05143,e0a779ee-1d88-4c2d-ae82-a2d4e6a90fd6
3,How do I create a simple IoT project with Rasp...,I don't have information on creating a simple ...,None,"For a simple IoT project with Raspberry Pi, st...",1,1.338315,9a0a4607-2b8b-47ea-a977-67a4360d9eec,fbbd447e-8e8f-44e6-8d09-5fb8983fdf0b
4,What's the difference between Arduino Uno and ...,The Arduino Uno is a microcontroller board tha...,None,Arduino Uno is a microcontroller board best fo...,1,2.487345,9d937a05-4c2d-49a4-9c53-c37c10df9c61,8eb0225a-2a5a-4074-b4ba-679549c5d854
5,What are common mistakes beginners make with A...,Common mistakes beginners make with Arduino in...,None,Common Arduino mistakes include: forgetting cu...,1,3.372534,a6f8deda-8043-47e1-b83c-6927398e288b,bd45aea5-fad2-49d1-875a-ef07720da1c4
6,How do I control a servo motor with Raspberry Pi?,"To control a servo motor with a Raspberry Pi, ...",None,"To control a servo motor with Raspberry Pi, co...",1,4.099423,b9bfaa82-8932-409b-8b96-c740de91ee67,eeac8d3d-b172-41b1-a29b-79610db442da
7,How do I set up a basic LED circuit with Ardui...,To set up a basic LED circuit with an Arduino ...,None,To set up a basic LED circuit with Arduino Uno...,1,4.275719,c4b3276a-2a1d-4a12-a688-8e116ad23b7b,2abe5cbc-b994-43bd-a392-72719314f4f3
8,What are the best sensors to start with for Ar...,I don't know.,None,The best beginner sensors for Arduino include:...,1,1.322636,ea00c58e-74a2-4de1-8396-353e25aa4b9e,226e9efd-b5a2-4a16-9d5b-c1a96fdf1ac7
9,What programming languages can I use with Rasp...,You can use multiple programming languages wit...,None,Raspberry Pi supports multiple programming lan...,1,2.996389,f8c359e3-1514-41a6-befd-ca15aa56b3cb,0844114f-6bef-4781-9c30-c741e05b8592
